In [1]:
#import
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import shap

C:\Users\noton\anaconda3\envs\TN5\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Reading dataset
df=pd.read_csv('MF.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Columns: 150 entries, Cate_name to Yield
dtypes: int64(148), object(2)
memory usage: 79.8+ KB


In [3]:
#Building descriptors
X = df.drop(columns=['Yield', 'Cate_name', 'P_name' ])
print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df['Yield'],columns=['Yield'])
y=y.astype(int)
print('---Objective---')
print(y.head())

---Descriptors---
   Cate_MF_9  Cate_MF_33  Cate_MF_58  Cate_MF_68  Cate_MF_80  Cate_MF_102  \
0          0           0           0           0           0            0   
1          0           0           0           0           0            0   
2          0           0           0           0           0            0   
3          0           0           0           0           0            0   
4          0           0           0           0           0            0   

   Cate_MF_105  Cate_MF_114  Cate_MF_125  Cate_MF_142  ...  P_MF_1154  \
0            0            0            0            0  ...          1   
1            0            0            0            0  ...          0   
2            0            0            0            0  ...          0   
3            0            0            0            0  ...          0   
4            0            0            0            1  ...          1   

   P_MF_1199  P_MF_1272  P_MF_1380  P_MF_1570  P_MF_1689  P_MF_1750  \
0        

In [4]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
best_model_parametors = []
data_cv_score = []
cum_imp=pd.Series(index = X.columns)
cum_imp.fillna(0, inplace=True)

#RF
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    param_grid = {"max_depth":[2,3,4,5],
                  'n_estimators':[1000,3000,5000]}
    reg_rf = GridSearchCV(RandomForestRegressor(random_state=0), param_grid=param_grid, cv=5, n_jobs=16)
    reg_rf.fit(X_train,y_train['Yield'])
    reg_best = reg_rf.best_estimator_
    y_pred1 = reg_best.predict(X_train)
    y_pred2 = reg_best.predict(X_test)
            
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.root_mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.root_mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)

    parametors = reg_rf.best_params_
    best_model_parametors.append(parametors)

    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)
    data_cv_score.append(reg_rf.best_score_)
    
    #cumulative importances 
    importances = pd.Series(reg_best.feature_importances_, index = X.columns)
    cum_imp += importances
    
    print('----------------------')
    print('seed:', seed)
    print("Best Model Parameter:",reg_rf.best_params_)
    print("Best Model Score:",reg_rf.best_score_)
    print('R2_test:', r2_test)
print('R2_train_means:', sum(data_r2_train)/10)
print('CV_score_means:', sum(data_cv_score)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
Best Model Parameter: {'max_depth': 3, 'n_estimators': 5000}
Best Model Score: 0.8179088361659094
R2_test: 0.7095174615793014
----------------------
seed: 1
Best Model Parameter: {'max_depth': 5, 'n_estimators': 5000}
Best Model Score: 0.7262804724087639
R2_test: 0.8382805598826046
----------------------
seed: 2
Best Model Parameter: {'max_depth': 2, 'n_estimators': 1000}
Best Model Score: 0.7700082191080861
R2_test: 0.8020543301933294
----------------------
seed: 3
Best Model Parameter: {'max_depth': 2, 'n_estimators': 3000}
Best Model Score: 0.7468727717298235
R2_test: 0.8169925402079661
----------------------
seed: 4
Best Model Parameter: {'max_depth': 4, 'n_estimators': 1000}
Best Model Score: 0.7979081214354135
R2_test: 0.7319468916131984
----------------------
seed: 5
Best Model Parameter: {'max_depth': 4, 'n_estimators': 1000}
Best Model Score: 0.802370689223387
R2_test: 0.735881513377802
----------------------
seed: 6
Best Model Parameter: {'max_d

In [5]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['RMSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])
data_cv_score_pd = pd.DataFrame(data=data_cv_score, columns=['cv_score'])
data_parametors = pd.DataFrame([best_model_parametors])

data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, 
                      data_RMSE_test_pd, data_MAE_test_pd, data_cv_score_pd],
                     axis=1, join='inner')

data_all.loc['mean'] = data_all.mean()
print(data_all)

data_all.to_csv('../../score/MF/RF_score.csv')

      r2_train  RMSE_train  MAE_train   r2_test  RMSE_test   MAE_test  \
0     0.940060    6.647686   4.352444  0.709517  19.491964  11.524010   
1     0.943896    7.122904   4.254628  0.838281  12.517251   8.249133   
2     0.862614   11.273663   6.984174  0.802054  13.835663   7.992750   
3     0.857967   11.509829   6.929200  0.816993  13.252950   8.399896   
4     0.946738    6.776230   4.284211  0.731947  16.882754  10.433955   
5     0.951189    6.286832   3.685293  0.735882  17.922667  11.303620   
6     0.874368   10.153951   6.342541  0.727014  18.125323  10.757334   
7     0.882358   10.607768   6.691659  0.735599  15.481662   9.954064   
8     0.878355   10.347066   6.231636  0.743401  15.924194  10.958767   
9     0.821850   13.436693   8.590952  0.915939   7.696365   5.084645   
mean  0.895939    9.416262   5.834674  0.775663  15.113079   9.465817   

      cv_score  
0     0.817909  
1     0.726280  
2     0.770008  
3     0.746873  
4     0.797908  
5     0.802371  
6   